# Features, Labels, Join

In [ ]:
import pandas as pd
import numpy as np

## A simple static feature

In [183]:
def gender_feature(filename='gender_female.csv'):
    """ Reads in a CSV, drops NAs, returns the DataFrame. """
    
    df = pd.read_csv(filename)
    df = df.dropna(how='any')
    
    return df

In [275]:
my_gender = gender_feature()
my_gender.head()

,entity_id,gender_female
0,309,1.0
1,324,1.0
2,48,1.0
3,210,0.0
4,180,1.0


## A temporal feature

In [223]:
def incident_aggregation(as_of_date, agg_col, date_col, time_delta, 
                         agg_funcs, filename='incidents.csv'):
    """
    Reads and aggregates a CSV file over a date range.
    
    Args:
        as_of_date (datetime): End of the aggregation window (excluded).
        agg_col (str): Name of the column for aggregation.
        date_col (str): Name of the column that gives knowledge dates for 
                        the values in agg_col.
        time_delta (pd.Timedelta): Time range. Gives the time 
                                   window preceding the as_of_date over
                                   which we aggregate.
        agg_funcs (dict): A dictionary that maps column names to functions.
                          The functions will be applied to the groupby, 
                          and the resulting dataframe contains columns
                          named like <key>_<timedelta>.
        fileame (str): Path to the CSV that should be aggregated. The 
                       CSV must contain an entity_id column, as well as 
                       the columns given by agg_col and date_col.
        
    Returns (pd.DataFrame): A dataframe, uniquely indexed by entity_id,
                            with columns that contain the aggregations
                            from agg_funcs. 
    """
    
    # read the CSV
    df = pd.read_csv(filename)
    df[date_col] = pd.to_datetime(df[date_col])
    
    # restrict to data in aggregation window
    df = df.loc[df[date_col] < as_of_date,:]
    df = df.loc[df[date_col] >= (as_of_date-time_delta),:]
    
    # add as_of_date to the index
    df['as_of_date'] = as_of_date
    df = df.set_index(['entity_id','as_of_date'])
    
    # just some formatting for naming the columns
    nice_timedelta_str = str(time_delta).replace('00:00:00','').replace(' ','')
    agg_funcs = {k+'_'+nice_timedelta_str: v for k,v in agg_funcs.items()}
    
    # aggregate by entity_id and apply the functions
    return df[agg_col].groupby(level=[0,1]).agg(agg_funcs)

In [224]:
my_agg = incident_aggregation(pd.to_datetime('2016-01-01'),
                             'incident_type',
                             'incident_date',
                              pd.Timedelta(365,'d'), 
                              {'count_neglects': lambda x: sum(x=='neglect_of_duty'),
                              'count_conduct': lambda x: sum(x=='conduct_unbecoming')})

In [225]:
my_agg.head()

,,count_conduct_365days,count_neglects_365days
entity_id,as_of_date,,
0,2016-01-01,2,1
1,2016-01-01,2,1
2,2016-01-01,1,0
3,2016-01-01,1,2
4,2016-01-01,3,0


## Binary labels

In [271]:
def label_aggregation(as_of_date, time_delta, filename='incidents.csv'):
    """ Find if an entity has a 'discipline' or 'conduct_unbecoming' incident
        that is decided as sustained.
    Args:
        as_of_date (datetime): Beginning of the aggregation window (included).
        time_delta (pd.Timedelta): Time range. Gives the time 
                                   window following the as_of_date over
                                   which we aggregate.
        filename (str): Path to the incidents CSV, which contains
                        entity_id, incident type and date, and 
                        decision with date.
    Returns (pd.Series):
        A boolean series, indexed uniquely by entity_id and as_of_date,
        giving if the entity had at least one sustained disciplinary
        or conduct_unbecoming event that fell within the time window.
    """
    
    # load the CSV
    df = pd.read_csv(filename, parse_dates=['incident_date','decision_date'])
        
    # restrict to incidents after the as_of_date
    df = df.loc[df.incident_date>=as_of_date,:]
    
    # restrict to decisions in the time window
    df = df.loc[df.decision_date<(as_of_date+time_delta),:]

    # add the as_of_date to the index
    df['as_of_date'] = as_of_date
    df = df.set_index(['entity_id','as_of_date'])
    
    # binarize
    df['adverse_incident'] = df.incident_type.isin(['discipline','conduct_unbecoming'])\
                             &(df.decision=='sustained')
    
    # aggregate and return
    return df.adverse_incident.groupby(level=[0,1]).max() > 0

In [272]:
my_labels = label_aggregation(pd.to_datetime('2016-01-01'), pd.Timedelta(90,'d'))

In [273]:
my_labels.head()

entity_id  as_of_date
2          2016-01-01     True
3          2016-01-01    False
8          2016-01-01    False
9          2016-01-01    False
10         2016-01-01    False
Name: adverse_incident, dtype: bool

## Joining Labels and Features

In [278]:
my_gender['as_of_date'] = pd.to_datetime('2016-01-01')
my_gender = my_gender.set_index(['entity_id','as_of_date'])

In [285]:
dataset = my_labels.to_frame().join(my_gender, how='left')\
                    .join(my_agg, how='left')
    
dataset.head()

,,adverse_incident,gender_female,count_conduct_365days,count_neglects_365days
entity_id,as_of_date,,,,
2,2016-01-01,True,0.0,1.0,0.0
3,2016-01-01,False,0.0,1.0,2.0
8,2016-01-01,False,1.0,3.0,2.0
9,2016-01-01,False,0.0,1.0,0.0
10,2016-01-01,False,0.0,1.0,2.0


## But clearly, some entities are missing...
Make a table of 'active' entities for the given date.

In [286]:
def active_officers(as_of_date, filename='patrol_duty.csv'):
    """Check if an officer is on patrol duty for the as_of_date. If ambiguous, drop
        that officer."""
    
    # read CSV
    df = pd.read_csv(filename, parse_dates=['start_date','end_date'])
    
    # check if as_of_date falls between start and end date of duty
    df['active'] = (df.start_date<=as_of_date)&(df.end_date>=as_of_date)
    
    # remove entities that have >1 entry (ambiguous)
    df = df.drop_duplicates(subset='entity_id', keep=False)
    
    df['as_of_date'] = as_of_date
    df = df.set_index(['entity_id','as_of_date'])
    
    return df.index

In [288]:
my_active = active_officers(pd.to_datetime('2016-01-01'))

We can now index into the dataset with our new entity list:

In [293]:
dataset.loc[my_active,:].head()

,,adverse_incident,gender_female,count_conduct_365days,count_neglects_365days
entity_id,as_of_date,,,,
0,2016-01-01,NaN,NaN,NaN,NaN
2,2016-01-01,True,0.0,1.0,0.0
3,2016-01-01,False,0.0,1.0,2.0
7,2016-01-01,NaN,NaN,NaN,NaN
8,2016-01-01,False,1.0,3.0,2.0


We need to coalesce / impute!